## Common

### Init

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import re

### Import Data

In [53]:
#gdp
gdp = pd.read_csv('./data/KRGDP_1.csv', skiprows=26, dtype={0:'int'})
gdp.columns = ['Year', 'GDP']
gdp.set_index('Year', inplace=True)

#gdp deflator
gdp_deflator = pd.read_csv('./data/KRGDPDeflator.csv', skiprows=26, dtype={0:'int'})
gdp_deflator.columns = ['Year', 'GDPDeflator']
gdp_deflator.set_index('Year', inplace=True)

# Labor Share
ls_oecd = pd.read_csv('./data/KRLS_OECD.csv', skiprows=25, dtype={0:'int'})
ls_oecd.columns = ['Year', 'LS']
ls_oecd.set_index('Year', inplace=True)

### Mainpulate Data

In [3]:
# UDF: change_colnames unflatten_cols
def change_colnames(df, colnames):
    df.columns = colnames
    return df

def unflatten_cols(df: pd.DataFrame, delim: str = ""):
    """Unflatten a single column level into multiple column levels.

    Args:
        delim: the delimiter to split on to identify the multiple column values.

    Returns:
        A copy of the dataframe with the new column levels.

    """
    new_cols = pd.MultiIndex.from_tuples([tuple(col.split(delim)) for col in df.columns])
    ndf = df.copy()
    ndf.columns = new_cols

    return ndf

def flatten_cols(df: pd.DataFrame, delim: str = ""):
    """Flatten multiple column levels of the DataFrame into a one column level.

    Args:
        delim: the delimiter between the column values.

    Returns:
        A copy of the dataframe with the new column names.

    """
    new_cols = [delim.join((col_lev for col_lev in tup if col_lev))
                for tup in df.columns.values]
    ndf = df.copy()
    ndf.columns = new_cols

    return ndf

## Capital

In Hsieh's calculation, captial appears in both primal and dual estimates.  
For primal estimation, $\hat{K}$ is estimated using GFCF data differenctiated by products. 
$$\begin{aligned}
\ \hat{K}_t \ = & \ \sum s_{k,j}\hat{K}_{j,t}\\
where:\\
\ \hat{K}_{j,t} \ = & \ I_{j,t} + K_{j,t-1}*(1-\delta_j)\\
\ K_{1993} \ = & \ \frac{I_{j,1993}}{g_j + \delta_j}\\
\ s_{k,j} \ = & \ \frac{r_j*K_j}{\sum r_j*K_j} \\
\ \frac{r_j}{p} \ = & \ \frac{p_j^k}{p}(i-\hat{p}_k+\delta_j) \\
\end{aligned}$$

$I_j$: Real investment in capital $j$  
$\delta_j$ : depreciation of capital $j$  
$g_j$: average growth in $I_j$ in the past 5 years

### Import Data

In [4]:
#Capital Breakdown
gfcf = pd.read_csv('./data/KRGFCF_breakdown.csv', index_col=[0]).dropna()
gfcf_real = pd.read_csv('./data/KRGFCF_breakdown_real.csv', index_col=[0]).dropna()

In [5]:
#Gross Capital
gfcf_all = pd.read_csv('./data/KRGFCF.csv', skiprows=25, dtype={0:'int'})
gfcf_all.columns = ['Year', 'GFCF']
gfcf_all = gfcf_all.set_index('Year')

gfcf_all_real = pd.read_csv('./data/KRGFCF_real.csv', skiprows=25, dtype={0:'int'})
gfcf_all_real.columns = ['Year', 'GFCFReal']
gfcf_all_real = gfcf_all_real.set_index('Year')

In [6]:
#Discount Rate
discount_rate = pd.read_csv('./data/KRDiscountRate.csv')
discount_rate.columns =['date', 'DiscountRate']

### Tidy Data

In [7]:
# unflatten colnames and set index
gfcf_real =\
(gfcf_real
 .rename(columns=lambda x: x.replace("GDP: 2015p: sa: GFCF: ", "").replace(" ", ""))
 .drop(['Construction(CS)', 'CS:Buildings', 'FacilitiesInvestment(FI)', 'IntellectualPropertyProducts(IPP)'], axis=1)
 .pipe(unflatten_cols, ":")
 .T
 .reset_index()
 .assign(level_2=lambda x: x.level_2.mask(lambda x: x.isna(), x.level_1))
 .set_index(['level_0', 'level_1', 'level_2'])
 .T
 .rename_axis(['Category', 'SubCategory1','SubCategory2'], axis=1)
 .rename_axis('Year', axis=0)
 #filter
 .loc[:,'CS':'FI']
 .droplevel(['Category', 'SubCategory1',], axis=1)
)


gfcf =\
(gfcf
 .rename(columns=lambda x: x.replace("GDP: sa: GFCF: ", "").replace(" ", ""))
 .drop(['Construction(CS)', 'CS:Buildings', 'FacilitiesInvestment(FI)', 'IntellectualPropertyProducts(IPP)'], axis=1)
 .pipe(unflatten_cols, ":")
  .T
 .reset_index()
 .assign(level_2=lambda x: x.level_2.mask(lambda y: y.isna(), x.level_1))
 .set_index(['level_0', 'level_1', 'level_2'])
 .T
 .rename_axis(['Category', 'SubCategory1','SubCategory2'], axis=1)
 .rename_axis('Year', axis=0)
 #filter
 .loc[:,'CS':'FI']
 .droplevel(['Category', 'SubCategory1',], axis=1)
)

In [8]:
# discount Rate
discount_rate =\
(discount_rate
 .assign(date=lambda x: pd.to_datetime(x.date))
 .assign(year=lambda x: x.date.dt.year,
         month=lambda x: x.date.dt.month)
 .query('month==12')
 .set_index('year')
 .rename_axis('Year', axis=0)
 ['DiscountRate']
 .apply(lambda x: x/100)
)

In [9]:
# Depreciation Rate
gfcf_delta = pd.Series(index=gfcf.T.index, data=[0.013, 0.029, 0.021, 0.182, 0.138])
gfcf_delta.name = 'Depreciation'

### Transform Data

In [10]:
#Capital Stock:First 5 year growth
gfcf_g =\
(gfcf_real
 .query('Year >= 1987 & Year <= 1992')
 .pct_change()
 .mean()
)

In [11]:
#real capital stock
gfcf_stock =\
(pd.DataFrame(gfcf_real.xs(1992)/(gfcf_g+gfcf_delta))
 .T
 .set_index([pd.Index([1992])], 'int')
 .rename_axis('Year', axis=0)
)

for year in np.arange(1993, 2019):
    year_stock = gfcf_real.xs(year) + gfcf_stock.xs(year-1)*(1-gfcf_delta)

    year_stock.name=year

    gfcf_stock = gfcf_stock.append(year_stock)

# real capital stock growth
gfcf_stock_growth = gfcf_stock.pct_change()

In [12]:
#RealRent: Prepare Vars
# Capital Inflation
pk_hat = (gfcf_all['GFCF']/gfcf_all_real['GFCFReal']).pct_change()
pk_hat.name = 'CapitalInflation'

#gfcf_deflactor
gfcf_deflactor = \
((gfcf/gfcf_real*100)
 .stack()
 .to_frame()
 .rename(columns={0:'GFCFDeflator'})
)

In [13]:
#Real Rent
gfcf_rent =\
(gfcf_deflactor
 .join(gdp_deflator)
 .assign(RelativePrice=lambda x: x.GFCFDeflator/x.GDPDeflator)
 .join(discount_rate)
 .join(pk_hat)
 .join(gfcf_delta)
 .assign(RealRent=lambda x: x.RelativePrice*(x.DiscountRate - x.CapitalInflation + x.Depreciation))
 .query('Year >= 1992')
 ['RealRent']
)

In [14]:
# Share
def get_gfcf_share(df):
    
    year_sum = df.groupby('Year')['TotalRent'].sum()
    
    df = df.assign(Share=lambda x: x.TotalRent/year_sum)
    
    return df

gfcf_share =\
(pd.concat([gfcf_stock.stack(),  gfcf_rent], axis=1)
 .rename(columns={0:'Stock'})
 .assign(TotalRent=lambda x: x.Stock * x.RealRent)
 .pipe(get_gfcf_share)
 ['Share']
)

gfcf_share_avg = gfcf_share.groupby('SubCategory2').mean()

#### $\hat{K}$

In [15]:
gfcf_stock_growth = (gfcf_stock.stack().groupby('SubCategory2').pct_change())

wkc =\
(gfcf_stock_growth.to_frame().join(gfcf_share_avg)
 .rename(columns={0:'CapitalStockGrowth'})
 .assign(wK_hat=lambda x: x.CapitalStockGrowth * x.Share)
 .groupby('Year')
 ['wK_hat']
 .sum()
)

#### $\hat{r}$

In [16]:
gfcf_rent_growth = gfcf_rent.groupby('SubCategory2').pct_change()
gfcf_rent_growth.name = 'RealRentGrowth'

wrc = \
(gfcf_rent_growth.to_frame().join(gfcf_share_avg)
 .assign(wr_hat=lambda x: x.RealRentGrowth*x.Share)
 .groupby('Year')
 ['wr_hat']
 .sum()
)

## Labor

### Import Data

In [17]:
# labor
labor93 = pd.read_csv("./data/KRLabor1.csv", index_col=list(range(0,6)), header=([0]))
labor00 = pd.read_csv("./data/KRLabor2.csv", index_col=list(range(0,6)), header=([0]))
labor09 = pd.read_csv("./data/KRLabor3.csv", index_col=list(range(0,6)), header=([0]))

### Tidy Data

In [18]:
# columns and format
def common_labor_tidy(df):
    df =\
    (df
     # tidy row and format
     .rename(columns=lambda x: re.sub('[^0-9]', "", x))
     .droplevel('UNIT')
     .unstack('Item')
     .reorder_levels([1,0], axis=1)
     .sort_index(axis=1)
     .rename_axis(['Item', 'Year'], axis=1)
     .stack('Year')
     .rename(columns=lambda x: x.replace(" ", ""))
     .reorder_levels([4,0,1,2,3])
     .sort_index(0)
     
     # tidy data
     .replace("-", np.NaN)
     .astype('float')
    )
    
    df.index = df.index.set_levels(df.index.levels[0].astype('int'), level=0)
    
    return df

In [19]:
# tidy frames: 93, 00, 09
labor93 = \
(labor93
 .pipe(common_labor_tidy)
)

labor00 =\
(labor00
 .pipe(common_labor_tidy)

)

labor09 =\
(labor09
 .pipe(common_labor_tidy)
 .pipe(change_colnames, ['Employment', 'Totalhoursworked', 'MonthlyTotalwage',])
 .rename_axis(index={'INDUSTRY_9S':'Industry'})
 # 09 data Monthly wage is 1/10000 of last 2 reports
 .assign(MonthlyTotalwage=lambda x: x.MonthlyTotalwage*1000)
)

In [20]:
# bind labor dfs

labor = pd.concat([labor93, labor00, labor09], axis=0).sort_index()

### Transform Data

In [21]:
# New Cols and Aggregated
# Columns: W: Hourly Wage, H: Man Hours Worked, TW: Total Wage Earned, i.e. labor cost.
# Aggregate: by 'Education', 'Sex', 'Age', 'Year'
labor_agg =\
(labor
 #filter
#  .query('Industry != "Agriculture,hunting and forestry(01-02)" &\
#          Industry != "Fishing(05)" &\
#          Industry != "Agriculture, forestry and fishing"')
 .query('Industry == "Mining and quarrying(10-14)" | Industry == "Mining and quarrying"')
 #mutate
 .assign(W=lambda x: round(x.MonthlyTotalwage, 0)/x.Totalhoursworked,
         H=lambda x: x.Employment*x.Totalhoursworked)
 #aggregate
 .groupby(['Education', 'Sex', 'Age', 'Year'])
 .agg({'W':'mean',
       'H': 'sum'})
 .assign(TW=lambda x: x.W*x.H)
)

In [22]:
# New Variables: Weight, GDPDeflaor
tw_yr =\
(labor_agg
 .groupby('Year')
 ['TW']
 .sum()
)



labor_agg =\
(labor_agg
 .assign(Share=lambda x: x.TW/tw_yr)
 .join(gdp_deflator)
)

In [23]:
# Growth in Variables
labor_agg_hat=\
(labor_agg
 .groupby(['Education', 'Sex', 'Age'])
 [['W', 'H', 'TW', 'GDPDeflator']]
 .pct_change()
 .rename(columns=lambda x: x + "_hat")
)

labor_agg =\
(labor_agg
 .join(labor_agg_hat)
)

In [24]:
# Weighted Wage Change and Weighted Labor Change

labor_agg =\
(labor_agg
 .assign(w_hat=lambda x: (1+x.W_hat)/(1+x.GDPDeflator_hat)-1)
 .assign(ww_hat=lambda x: x.Share*x.w_hat)
 .assign(wH_hat=lambda x: x.Share*x.H_hat)
 .replace(np.inf, np.NaN)
)


wwc =\
(labor_agg
 .groupby('Year')
 ['ww_hat']
 .sum()
 .drop(2009)
 .replace(np.inf, np.NaN)
)

whc =\
(labor_agg
 .groupby('Year')
 ['wH_hat']
 .sum()
 .drop(2009)
)

In [25]:
import numpy as np
import pandas as pd

## Primal

In [56]:
# Y hat
yhat =\
(gdp.pct_change()['GDP'] -
 gdp_deflator.pct_change()['GDPDeflator']
)

yhat.name = 'y_hat'

In [59]:
primal =\
(pd.concat([whc,wkc,ls_oecd, yhat], axis=1, join='inner')
 .assign(LS=lambda x: x.LS/100,
         KS=lambda x: 1-x.LS )
 .assign(Primal=lambda x: x.y_hat - x.KS*x.wK_hat - x.LS*x.wH_hat)
)

In [63]:
px.scatter(primal.reset_index().melt(id_vars='Year'), x='Year', y='value', color='variable', trendline='ols')

## Dual

In [69]:
dual =\
(pd.concat([wrc, wwc, ls_oecd], axis=1, join='inner')
 .assign(LS=lambda x: x.LS/100,
         KS=lambda x: 1-x.LS )
 .assign(Dual=lambda x: x.KS*x.wr_hat + x.LS*x.ww_hat)
)

In [70]:
px.scatter(dual.reset_index().melt(id_vars='Year'), x='Year', y='value', color='variable', trendline='ols')

In [72]:
tfpg = pd.concat([primal['Primal'], dual['Dual']], axis=1)

In [74]:
px.line(tfpg.reset_index().melt(id_vars='Year'), x='Year', y='value', color='variable')